### Intro

This notebook explores the model we made to determine Titanic survival/death probability based on a number of variables.

# Libraries

The libraries are used for various tasks such as web development (Flask), data manipulation (pandas), visualization (seaborn), machine learning (scikit-learn), and numerical computations (numpy).

In [ ]:
from flask import Flask, request, jsonify, Blueprint
from flask_restful import Api, Resource
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np

# Blueprint/Resoucre Creation

In [ ]:
titanic_api = Blueprint('titanic_api', __name__, url_prefix='/api/titanic')
api = Api(titanic_api)

# TitanicAPI Class

### Initialization

--> Initialize machine learning model and prepare the Titanic dataset.

In [2]:
class TitanicAPI(Resource):
    def __init__(self):
        # Load the Titanic dataset
        titanic_data = sns.load_dataset('titanic')
        
        # Preprocess the dataset
        td = titanic_data.copy()
        td.drop(['alive', 'who', 'adult_male', 'class', 'embark_town', 'deck'], axis=1, inplace=True)
        td.dropna(inplace=True)
        td['sex'] = td['sex'].apply(lambda x: 1 if x == 'male' else 0)
        td['alone'] = td['alone'].apply(lambda x: 1 if x else 0)

        # Encode categorical variables
        self.enc = OneHotEncoder(handle_unknown='ignore')
        embarked_encoded = self.enc.fit_transform(td[['embarked']].values.reshape(-1, 1))
        self.encoded_cols = self.enc.get_feature_names_out(['embarked'])
        td[self.encoded_cols] = embarked_encoded.toarray()
        td.drop(['embarked'], axis=1, inplace=True)

        # Initialize Logistic Regression model
        self.logreg = LogisticRegression(max_iter=1000)
        X = td.drop('survived', axis=1)
        y = td['survived']
        self.logreg.fit(X, y)

NameError: name 'Resource' is not defined

### predict_survival Method

Takes passenger data as input, preprocesses it --> uses the trained Logistic Regression model to predict survival probabilities.

In [ ]:
    def predict_survival(self, data):
        try:
            # Preprocess input data
            passenger = pd.DataFrame([data]) 
            passenger['sex'] = passenger['sex'].apply(lambda x: 1 if x == 'male' else 0)
            passenger['alone'] = passenger['alone'].apply(lambda x: 1 if x else 0)
            embarked_encoded = self.enc.transform(passenger[['embarked']].values.reshape(-1, 1))
            passenger[self.encoded_cols] = embarked_encoded.toarray()
            passenger.drop(['embarked', 'name'], axis=1, inplace=True)

            # Predict survival probabilities
            dead_proba, alive_proba = np.squeeze(self.logreg.predict_proba(passenger))

            return {
                'Death probability': '{:.2%}'.format(dead_proba),
                'Survival probability': '{:.2%}'.format(alive_proba)
            }
        except Exception as e:
            return {'error': str(e)}

### Endpoint

- Define a POST endpoint /predict to receive passenger data --> return survival predictions.
- Endpoint essentially also handles exceptions and returns appropriate responses.

In [1]:
    def post(self):
        try:
            data = request.json
            result = self.predict_survival(data)
            return jsonify(result)
        except Exception as e:
            return jsonify({'error': str(e)})

api.add_resource(TitanicAPI, '/predict')

NameError: name 'api' is not defined